In [1]:
import os

import openmm as mm
from openmm.app import *
from openmm import unit
from openmm import app
from pdbfixer import PDBFixer
import mdtraj as md
import nglview as nv
import requests
import numpy as np
from math import sqrt
from openmmtools import testsystems

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.


In [2]:
alanine_implicit = testsystems.AlanineDipeptideImplicit(constraints=HBonds)

In [102]:
alanine_implicit.positions

Quantity(value=array([[ 2.0000010e-01,  1.0000000e-01, -1.3000000e-07],
       [ 2.0000010e-01,  2.0900000e-01,  1.0000000e-08],
       [ 1.4862640e-01,  2.4538490e-01,  8.8982400e-02],
       [ 1.4862590e-01,  2.4538520e-01, -8.8982000e-02],
       [ 3.4274200e-01,  2.6407950e-01, -3.0000000e-07],
       [ 4.3905800e-01,  1.8774060e-01, -6.6000000e-07],
       [ 3.5553754e-01,  3.9696488e-01, -3.1000000e-07],
       [ 2.7331200e-01,  4.5561601e-01, -1.3000000e-07],
       [ 4.8532621e-01,  4.6139253e-01, -4.3000000e-07],
       [ 5.4075960e-01,  4.3155388e-01,  8.8981520e-02],
       [ 5.6613044e-01,  4.2208425e-01, -1.2321480e-01],
       [ 5.1232615e-01,  4.5213630e-01, -2.1312016e-01],
       [ 6.6304840e-01,  4.7189354e-01, -1.2057907e-01],
       [ 5.8085401e-01,  3.1408724e-01, -1.2413850e-01],
       [ 4.7126759e-01,  6.1294185e-01,  1.4000000e-07],
       [ 3.6006445e-01,  6.6527027e-01,  6.2000000e-07],
       [ 5.8460533e-01,  6.8348833e-01,  2.5000000e-07],
       [ 6.73700

In [3]:
system = alanine_implicit.system

In [82]:
system.getNumForces()

6

In [85]:
system.getForces()

[<openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7f140036fcf0> >,
 <openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7f140036f660> >,
 <openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x7f140036fb40> >,
 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7f140036fe10> >,
 <openmm.openmm.CustomGBForce; proxy of <Swig Object of type 'OpenMM::CustomGBForce *' at 0x7f140036f9c0> >,
 <openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x7f140036ffc0> >]

In [4]:
topology = alanine_implicit.topology
topology

<Topology; 1 chains, 3 residues, 22 atoms, 21 bonds>

In [76]:
alanine_implicit.analytical_properties?

Type:        property
String form: <property object at 0x7f1420fac6b0>
Docstring:   A list of available analytical properties, accessible via 'get_propertyname(thermodynamic_state)' calls.


In [5]:
properties = alanine_implicit.analytical_properties
properties

[]

In [6]:
implicit = system.getForce(4)
implicit

<openmm.openmm.CustomGBForce; proxy of <Swig Object of type 'OpenMM::CustomGBForce *' at 0x7f14c85b80f0> >

In [21]:
implicit.getComputedValueParameters?

Signature: implicit.getComputedValueParameters(index)
Docstring:
getComputedValueParameters(self, index)
Get the properties of a computed value.

Parameters
----------
index : int
    the index of the computed value for which to get parameters

Returns
-------
name : string
    the name of the value
expression : string
    an algebraic expression to evaluate when calculating the computed value. If the ComputationType is SingleParticle, the expression is evaluated independently for each particle, and may depend on its x, y, and z coordinates, as well as the per-particle parameters and previous computed values for that particle. If the ComputationType is ParticlePair or ParticlePairNoExclusions, the expression is evaluated once for every other particle in the system and summed to get the final value. In the latter case, the expression may depend on the distance r between the two particles, and on the per-particle parameters and previous computed values for each of them. Append "1" to a v

In [7]:
implicit.getComputedValueParameters(0)

['I',
 'select(step(r+sr2-or1), 0.5*(1/L-1/U+0.25*(r-sr2^2/r)*(1/(U^2)-1/(L^2))+0.5*log(L/U)/r), 0);U=r+sr2;L=max(or1, D);D=abs(r-sr2)',
 2]

In [8]:
implicit.getComputedValueParameters(1)

['B',
 '1/(1/or-tanh(0.8*psi+2.909125*psi^3)/radius);psi=I*or; radius=or+offset; offset=0.009',
 0]

In [22]:
implicit.getCutoffDistance?

Signature: implicit.getCutoffDistance()
Docstring:
getCutoffDistance(self) -> double
Get the cutoff distance (in nm) being used for nonbonded interactions. If the NonbondedMethod in use is NoCutoff, this value will have no effect.

Returns
-------
double
    the cutoff distance, measured in nm
File:      ~/MyOpt/Miniconda/envs/bifree/lib/python3.7/site-packages/openmm/openmm.py
Type:      method


In [9]:
implicit.getCutoffDistance()

Quantity(value=1.0, unit=nanometer)

In [10]:
implicit.getEnergyParameterDerivativeName?

Signature: implicit.getEnergyParameterDerivativeName(index)
Docstring:
getEnergyParameterDerivativeName(self, index) -> std::string const &
Get the name of a global parameter with respect to which this Force should compute the derivative of the energy.

Parameters
----------
index : int
    the index of the parameter derivative, between 0 and getNumEnergyParameterDerivatives()

Returns
-------
string
    the parameter name
File:      ~/MyOpt/Miniconda/envs/bifree/lib/python3.7/site-packages/openmm/openmm.py
Type:      method


In [11]:
implicit.getEnergyParameterDerivativeName(0)

OpenMMException: Assertion failure at CustomGBForce.cpp:126.  Index out of range

In [12]:
implicit.getEnergyTermParameters?

Signature: implicit.getEnergyTermParameters(index)
Docstring:
getEnergyTermParameters(self, index)
Get the properties of a term to the energy computation.

Parameters
----------
index : int
    the index of the term for which to get parameters

Returns
-------
expression : string
    an algebraic expression to evaluate when calculating the energy. If the ComputationType is SingleParticle, the expression is evaluated once for each particle, and may depend on its x, y, and z coordinates, as well as the per-particle parameters and computed values for that particle. If the ComputationType is ParticlePair or ParticlePairNoExclusions, the expression is evaluated once for every pair of particles in the system. In the latter case, the expression may depend on the distance r between the two particles, and on the per-particle parameters and computed values for each of them. Append "1" to a variable name to indicate the parameter for the first particle in the pair and "2" to indicate the second p

In [13]:
implicit.getEnergyTermParameters(0)

['-0.5*138.935485*(1/soluteDielectric-1/solventDielectric)*charge^2/B; solventDielectric=78.5; soluteDielectric=1; kappa=0; offset=0.008999999999999999',
 0]

In [14]:
implicit.getEnergyTermParameters(1)

['28.3919551*(radius+0.14)^2*(radius/B)^6; radius=or+offset; solventDielectric=78.5; soluteDielectric=1; kappa=0; offset=0.008999999999999999',
 0]

In [15]:
implicit.getEnergyTermParameters(2)

['-138.935485*(1/soluteDielectric-1/solventDielectric)*charge1*charge2/f;f=sqrt(r^2+B1*B2*exp(-r^2/(4*B1*B2))); solventDielectric=78.5; soluteDielectric=1; kappa=0; offset=0.008999999999999999',
 2]

In [16]:
implicit.getExclusionParticles?

Signature: implicit.getExclusionParticles(index)
Docstring:
getExclusionParticles(self, index)
Get the particles in a pair whose interaction should be excluded.

Parameters
----------
index : int
    the index of the exclusion for which to get particle indices

Returns
-------
particle1 : int
    the index of the first particle in the pair
particle2 : int
    the index of the second particle in the pair
File:      ~/MyOpt/Miniconda/envs/bifree/lib/python3.7/site-packages/openmm/openmm.py
Type:      method


In [17]:
implicit.getForceGroup?

Signature: implicit.getForceGroup()
Docstring:
getForceGroup(self) -> int
Get the force group this Force belongs to.
File:      ~/MyOpt/Miniconda/envs/bifree/lib/python3.7/site-packages/openmm/openmm.py
Type:      method


In [18]:
implicit.getForceGroup()

0

In [19]:
implicit.getFunctionParameters?

Signature: implicit.getFunctionParameters(index)
Docstring:
getFunctionParameters(self, index)
Get the parameters for a tabulated function that may appear in expressions.

 @deprecated This method exists only for backward compatibility. Use getTabulatedFunctionParameters() instead. If the specified function is not a Continuous1DFunction, this throws an exception.
File:      ~/MyOpt/Miniconda/envs/bifree/lib/python3.7/site-packages/openmm/openmm.py
Type:      method


In [23]:
implicit.getGlobalParameterDefaultValue?

Signature: implicit.getGlobalParameterDefaultValue(index)
Docstring:
getGlobalParameterDefaultValue(self, index) -> double
Get the default value of a global parameter.

Parameters
----------
index : int
    the index of the parameter for which to get the default value

Returns
-------
double
    the parameter default value
File:      ~/MyOpt/Miniconda/envs/bifree/lib/python3.7/site-packages/openmm/openmm.py
Type:      method


In [29]:
implicit.getGlobalParameterDefaultValue(0)

OpenMMException: Assertion failure at CustomGBForce.cpp:107.  Index out of range

In [30]:
implicit.getGlobalParameterName?

Signature: implicit.getGlobalParameterName(index)
Docstring:
getGlobalParameterName(self, index) -> std::string const &
Get the name of a global parameter.

Parameters
----------
index : int
    the index of the parameter for which to get the name

Returns
-------
string
    the parameter name
File:      ~/MyOpt/Miniconda/envs/bifree/lib/python3.7/site-packages/openmm/openmm.py
Type:      method


In [89]:
implicit.getGlobalParameterDefaultValue()

TypeError: getGlobalParameterDefaultValue() missing 1 required positional argument: 'index'

In [32]:
implicit.getName?

Signature: implicit.getName()
Docstring:
getName(self) -> std::string const &
Get the name of this Force. This is an arbitrary, user modifiable identifier. By default it equals the class name, but you can change it to anything useful.
File:      ~/MyOpt/Miniconda/envs/bifree/lib/python3.7/site-packages/openmm/openmm.py
Type:      method


In [33]:
implicit.getName()

'CustomGBForce'

In [34]:
implicit.getNonbondedMethod?

Signature: implicit.getNonbondedMethod()
Docstring:
getNonbondedMethod(self) -> OpenMM::CustomGBForce::NonbondedMethod
Get the method used for handling long range nonbonded interactions.
File:      ~/MyOpt/Miniconda/envs/bifree/lib/python3.7/site-packages/openmm/openmm.py
Type:      method


In [35]:
implicit.getNonbondedMethod()

0

In [36]:
implicit.getNumComputedValues?

Signature: implicit.getNumComputedValues()
Docstring:
getNumComputedValues(self) -> int
Get the number of per-particle computed values the interaction depends on.
File:      ~/MyOpt/Miniconda/envs/bifree/lib/python3.7/site-packages/openmm/openmm.py
Type:      method


In [37]:
implicit.getNumComputedValues()

2

In [38]:
implicit.getNumEnergyParameterDerivatives?

Signature: implicit.getNumEnergyParameterDerivatives()
Docstring:
getNumEnergyParameterDerivatives(self) -> int
Get the number of global parameters with respect to which the derivative of the energy should be computed.
File:      ~/MyOpt/Miniconda/envs/bifree/lib/python3.7/site-packages/openmm/openmm.py
Type:      method


In [40]:
implicit.getNumEnergyParameterDerivatives()

0

In [41]:
implicit.getNumEnergyTerms?

Signature: implicit.getNumEnergyTerms()
Docstring:
getNumEnergyTerms(self) -> int
Get the number of terms in the energy computation.
File:      ~/MyOpt/Miniconda/envs/bifree/lib/python3.7/site-packages/openmm/openmm.py
Type:      method


In [42]:
implicit.getNumEnergyTerms()

3

In [43]:
implicit.getNumExclusions?

Signature: implicit.getNumExclusions()
Docstring:
getNumExclusions(self) -> int
Get the number of particle pairs whose interactions should be excluded.
File:      ~/MyOpt/Miniconda/envs/bifree/lib/python3.7/site-packages/openmm/openmm.py
Type:      method


In [44]:
implicit.getNumExclusions()

0

In [45]:
implicit.getNumFunctions?

Signature: implicit.getNumFunctions()
Docstring:
getNumFunctions(self) -> int
Get the number of tabulated functions that have been defined.

 @deprecated This method exists only for backward compatibility. Use getNumTabulatedFunctions() instead.
File:      ~/MyOpt/Miniconda/envs/bifree/lib/python3.7/site-packages/openmm/openmm.py
Type:      method


In [46]:
implicit.getNumFunctions()

0

In [47]:
implicit.getNumGlobalParameters?

Signature: implicit.getNumGlobalParameters()
Docstring:
getNumGlobalParameters(self) -> int
Get the number of global parameters that the interaction depends on.
File:      ~/MyOpt/Miniconda/envs/bifree/lib/python3.7/site-packages/openmm/openmm.py
Type:      method


In [48]:
implicit.getNumGlobalParameters()

0

In [49]:
implicit.getNumParticles?

Signature: implicit.getNumParticles()
Docstring:
getNumParticles(self) -> int
Get the number of particles for which force field parameters have been defined.
File:      ~/MyOpt/Miniconda/envs/bifree/lib/python3.7/site-packages/openmm/openmm.py
Type:      method


In [50]:
implicit.getNumParticles()

22

In [51]:
implicit.getNumPerParticleParameters?

Signature: implicit.getNumPerParticleParameters()
Docstring:
getNumPerParticleParameters(self) -> int
Get the number of per-particle parameters that the interaction depends on.
File:      ~/MyOpt/Miniconda/envs/bifree/lib/python3.7/site-packages/openmm/openmm.py
Type:      method


In [52]:
implicit.getNumPerParticleParameters()

3

In [53]:
implicit.getNumTabulatedFunctions?

Signature: implicit.getNumTabulatedFunctions()
Docstring:
getNumTabulatedFunctions(self) -> int
Get the number of tabulated functions that have been defined.
File:      ~/MyOpt/Miniconda/envs/bifree/lib/python3.7/site-packages/openmm/openmm.py
Type:      method


In [54]:
implicit.getNumTabulatedFunctions()

0

In [55]:
implicit.getParticleParameters?

Signature: implicit.getParticleParameters(index)
Docstring:
getParticleParameters(self, index)
Get the nonbonded force parameters for a particle.

Parameters
----------
index : int
    the index of the particle for which to get parameters

Returns
-------
parameters : vector< double >
    the list of parameters for the specified particle
File:      ~/MyOpt/Miniconda/envs/bifree/lib/python3.7/site-packages/openmm/openmm.py
Type:      method


In [66]:
implicit.getParticleParameters(0)

(0.1123, 0.111, 0.09435)

In [65]:
for ii in range(implicit.getNumParticles()):
    print(implicit.getParticleParameters(ii))

(0.1123, 0.111, 0.09435)
(-0.3662, 0.16099999999999998, 0.11591999999999998)
(0.1123, 0.111, 0.09435)
(0.1123, 0.111, 0.09435)
(0.5972000021951126, 0.16099999999999998, 0.11591999999999998)
(-0.5679000016463344, 0.141, 0.11984999999999998)
(-0.4157, 0.146, 0.11534)
(0.27190000000000003, 0.12100000000000001, 0.10285000000000001)
(0.033699999999999994, 0.16099999999999998, 0.11591999999999998)
(0.0823, 0.111, 0.09435)
(-0.1825, 0.16099999999999998, 0.11591999999999998)
(0.06029999999999999, 0.111, 0.09435)
(0.06029999999999999, 0.111, 0.09435)
(0.06029999999999999, 0.111, 0.09435)
(0.5973000005487781, 0.16099999999999998, 0.11591999999999998)
(-0.5679000016463344, 0.141, 0.11984999999999998)
(-0.4157, 0.146, 0.11534)
(0.27190000000000003, 0.12100000000000001, 0.10285000000000001)
(-0.149, 0.16099999999999998, 0.11591999999999998)
(0.0976, 0.111, 0.09435)
(0.0976, 0.111, 0.09435)
(0.0976, 0.111, 0.09435)


In [67]:
implicit.getPerParticleParameterName?

Signature: implicit.getPerParticleParameterName(index)
Docstring:
getPerParticleParameterName(self, index) -> std::string const &
Get the name of a per-particle parameter.

Parameters
----------
index : int
    the index of the parameter for which to get the name

Returns
-------
string
    the parameter name
File:      ~/MyOpt/Miniconda/envs/bifree/lib/python3.7/site-packages/openmm/openmm.py
Type:      method


In [68]:
implicit.getPerParticleParameterName(0)

'charge'

In [69]:
for uu in range(implicit.getNumPerParticleParameters()):
    print(implicit.getPerParticleParameterName(uu))

charge
or
sr


In [70]:
implicit.getTabulatedFunction?

Signature: implicit.getTabulatedFunction(*args)
Docstring:
getTabulatedFunction(self, index) -> TabulatedFunction
getTabulatedFunction(self, index) -> TabulatedFunction
Get a reference to a tabulated function that may appear in expressions.

Parameters
----------
index : int
    the index of the function to get

Returns
-------
TabulatedFunction
    the TabulatedFunction object defining the function
File:      ~/MyOpt/Miniconda/envs/bifree/lib/python3.7/site-packages/openmm/openmm.py
Type:      method


In [71]:
implicit.getTabulatedFunction(0)

OpenMMException: Assertion failure at CustomGBForce.cpp:208.  Index out of range

In [93]:
implicit.getTabulatedFunctionName(2)

OpenMMException: Assertion failure at CustomGBForce.cpp:213.  Index out of range

In [94]:
implicit.get

3

In [74]:
implicit.getTabulatedFunctionName(0)

OpenMMException: Assertion failure at CustomGBForce.cpp:213.  Index out of range